## Nama : Sulthan Zaidan Fauzi<br>NPM : 1908107010028

### Tugas 1 - Versi 1

Tampilan awal akan blur. Ketika digambar sebuah kotak maka daerah pada kotak tersebut akan menjadi normal kembali (tidak blur lagi).

In [2]:
import cv2

def draw_rectangle(event, x, y, flags, params):
    global x_init, y_init, drawing, top_left_pt, bottom_right_pt
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x_init, y_init = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            top_left_pt = (min(x_init, x), min(y_init, y))
            bottom_right_pt = (max(x_init, x), max(y_init, y))
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        top_left_pt = (min(x_init, x), min(y_init, y))
        bottom_right_pt = (max(x_init, x), max(y_init, y))

if __name__=='__main__':
    drawing = False
    top_left_pt, bottom_right_pt = (-1,-1), (-1,-1)
    
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
        
    cv2.namedWindow('Webcam')
    cv2.setMouseCallback('Webcam', draw_rectangle)
    
    while True:
        ret, frame = cap.read()
        img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        
        (x0,y0), (x1,y1) = top_left_pt, bottom_right_pt
        
        # menggambar kotak hijau
        cv2.line(img, (x0,y0), (x0, y1), (0,255,0), 2)
        cv2.line(img, (x0,y1), (x1, y1), (0,255,0), 2)
        cv2.line(img, (x1,y1), (x1, y0), (0,255,0), 2)
        cv2.line(img, (x1,y0), (x0, y0), (0,255,0), 2)
        
        # blur semuanya kecuali daerah yang dipilih
        not_blur = img[y0:y1, x0:x1]
        #img = cv2.medianBlur(img, 15)
        img = cv2.blur(img, (30,30), 0)
        img[y0:y1, x0:x1] = not_blur
        
        cv2.imshow('Webcam', img)
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


### Tugas 1 - Versi 2

Tampilan akan terbagi menjadi 4 bagian sesuai dengan posisi pointer mouse.<br>
Bagian kiri atas menampilkan efek negatif.<br>
Bagian kanan atas menampilkan tampilan normal.<br>
Bagian kiri bawah menampilkan efek blur.<br>
Bagian kanan bawah menampilkan efek HSV.<br>

In [5]:
import cv2

def mouse(event, x, y, flags, params):
    global x_init, y_init
    
    if event == cv2.EVENT_MOUSEMOVE:
        x_init, y_init = x, y

if __name__=='__main__':
    x_init, y_init = -1, -1
    
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
        
    cv2.namedWindow('Webcam')
    cv2.setMouseCallback('Webcam', mouse)
    
    while True:
        ret, frame = cap.read()
        img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        
        # kiri-atas (negative)
        img[0:y_init, 0:x_init] = 255 - img[0:y_init, 0:x_init]
        
        # kanan-atas (normal)
        #img[0:y_init, x_init:img.shape[1]] =
        
        # kiri-bawah (median blur)
        if y_init < img.shape[0] - 10 and x_init > 10:
            img[y_init:img.shape[0], 0:x_init] = cv2.medianBlur(img[y_init:img.shape[0], 0:x_init], 15)
        
        # kanan-bawah (HSV)
        img[y_init:img.shape[0], x_init:img.shape[1]] = cv2.cvtColor(img[y_init:img.shape[0], x_init:img.shape[1]], cv2.COLOR_BGR2HSV)

        # garis hijau pemisah sesuai posisi pointer mouse
        cv2.line(img, (x_init,0), (x_init, img.shape[0]), (0,255,0), 2)
        cv2.line(img, (0,y_init), (img.shape[1], y_init), (0,255,0), 2)
            
        cv2.imshow('Webcam', img)
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
